In [1]:
path = '/Users/kanoumotoharu/Downloads/m5-forecasting-accuracy/'
#path = '/Users/abcdm/Downloads/m5-forecasting-accuracy/'
#path = '../input/m5-forecasting-accuracy/'

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, gc
import termcolor

import math, random
import pickle
import datetime, time
from tqdm import tqdm_notebook as tqdm

import torch 
from torch import nn
from torch import optim

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.cluster import KMeans

%matplotlib inline

In [3]:
df = pd.read_csv(path+'sales_train_validation.csv')
calendar_df = pd.read_csv(path+'calendar.csv')
sell_prices_df = pd.read_csv(path+'sell_prices.csv')
sample_submission_df = pd.read_csv(path+'sample_submission.csv')

In [14]:
df['index'] = df.index

In [4]:
'item_id_state_id_'

'item_id_state_id_'

In [6]:
d_cols = df.columns[df.columns.str.startswith('d_')]

In [7]:
d_cols

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1913)

In [32]:
dic = df.groupby(['item_id', 'state_id'])[d_cols[:100]].sum()#.T#.to_dict()

In [39]:
idx = dic.index

In [40]:
indexs = df.groupby(['item_id', 'state_id'])['index'].unique()

In [42]:
indexs[idx]

item_id          state_id
FOODS_1_001      CA          [1612, 4661, 7710, 10759]
                 TX              [13808, 16857, 19906]
                 WI              [22955, 26004, 29053]
FOODS_1_002      CA          [1613, 4662, 7711, 10760]
                 TX              [13809, 16858, 19907]
                 WI              [22956, 26005, 29054]
FOODS_1_003      CA          [1614, 4663, 7712, 10761]
                 TX              [13810, 16859, 19908]
                 WI              [22957, 26006, 29055]
FOODS_1_004      CA          [1615, 4664, 7713, 10762]
                 TX              [13811, 16860, 19909]
                 WI              [22958, 26007, 29056]
FOODS_1_005      CA          [1616, 4665, 7714, 10763]
                 TX              [13812, 16861, 19910]
                 WI              [22959, 26008, 29057]
FOODS_1_006      CA          [1617, 4666, 7715, 10764]
                 TX              [13813, 16862, 19911]
                 WI              [22960

In [29]:
len(indexs.values)

9147

In [23]:
index_len = df.groupby(['item_id', 'state_id'])['index'].nunique()
pd.DataFrame(index_len)

index
item_id         state_id       
FOODS_1_001     CA            4
                TX            3
                WI            3
FOODS_1_002     CA            4
                TX            3
                WI            3
FOODS_1_003     CA            4
                TX            3
                WI            3
FOODS_1_004     CA            4
                TX            3
                WI            3
FOODS_1_005     CA            4
                TX            3
                WI            3
FOODS_1_006     CA            4
                TX            3
                WI            3
FOODS_1_008     CA            4
                TX            3
                WI            3
FOODS_1_009     CA            4
                TX            3
                WI            3
FOODS_1_010     CA            4
                TX            3
                WI            3
FOODS_1_011     CA            4
                TX            3
                WI            3
...                         ...
HOUSEHOLD_2_507 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_508 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_509 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_510 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_511 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_512 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_513 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_514 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_515 CA            4
                TX            3
                WI            3
HOUSEHOLD_2_516 CA            4
                TX            3
                WI            3

[9147 rows x 1 columns]

In [ ]:
def create_objetive_index(original_df, group_id):
    df = original_df.copy()
    df['all_id'] = 0
    d_cols = df.columns[df.columns.str.startswith('d_')]
    df['index'] = df.index
    id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'all_id', 'index']
    denominator = {}
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
        for key, value in tmp_a.items() :
            value = np.array(list(value.values()))
            value = ((value[1:]-value[:-1])**2).mean()
            if value<=0:
                value=1
            a[key] = value
        denominator.update(a)

In [4]:
df['all_id'] = 0
d_cols = df.columns[df.columns.str.startswith('d_')]
group_ids = (
    ['all_id'],
    ['state_id'],
    ['store_id'],
    ['cat_id'],
    ['dept_id'],
    ['state_id', 'cat_id'],
    ['state_id', 'dept_id'],
    ['store_id', 'cat_id'],
    ['store_id', 'dept_id'],
    ['item_id'],
    ['item_id', 'state_id'],
    ['item_id', 'store_id']
)
df['index'] = df.index
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'all_id', 'index']

In [5]:
def get_weight(train_df, weight_columns, calendar, price_df, id_columns):
    day_to_week = calendar.set_index('d')['wm_yr_wk'].to_dict()
    weight_df = train_df[['item_id', 'store_id'] + weight_columns].set_index(['item_id', 'store_id'])
    weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
    weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)
    
    weight_df = weight_df.merge(price_df, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
    weight_df['value'] = weight_df['value'] * weight_df['sell_price']
    weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
    weight_df = weight_df.loc[zip(train_df.item_id, train_df.store_id), :].reset_index(drop=True)
    weight_df = pd.concat([train_df[id_columns], weight_df], axis=1, sort=False)        
    return weight_df

In [6]:
i=0
train_d_cols = d_cols[:-28*(i+1)]
valid_d_cols = d_cols[-28*(i+1):-28*i]
weight_d_cols = train_d_cols[-28:]
train_df = df[id_columns+train_d_cols]
val_df = df[id_columns+valid_d_cols]

weight_df = get_weight(
    train_df=train_df, weight_columns=weight_d_cols, 
    calendar=calendar_df, price_df=sell_prices_df, id_columns=id_columns
)

weight_data = {}
denominator = {}
name = []
for i, group_id in enumerate(tqdm(group_ids)):
    lv_weight = weight_df.groupby(group_id)[weight_d_cols].sum().sum(axis=1)
    weight_data.update((lv_weight / lv_weight.sum()).to_dict())
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
    for key, value in tmp_a.items() :
        value = np.array(list(value.values()))
        value = ((value[1:]-value[:-1])**2).mean()
        if value<=0:
            value=1
        a[key] = value
    denominator.update(a)
    
    s = str()
    for g in group_id:
        s+=f'{g}_'
    name+=[s for _ in range(len(a))]
    
    
    
    
weight_data = pd.DataFrame(pd.Series(weight_data))
weight_data.columns=['weight']
weight_data['group_id'] = name

denominator = pd.DataFrame(pd.Series(denominator))
denominator.columns=['denominator']
denominator = 1/denominator
denominator = np.sqrt(denominator)

wrm_param_df = pd.concat([weight_data, denominator], axis=1)
wrm_param_df['param'] = wrm_param_df.weight*wrm_param_df.denominator

wrm_param_df.sort_values('param', ascending=False, inplace=True)
    
    
priority_df = pd.concat([
    wrm_param_df.groupby(['group_id'])['param'].sum(),
    wrm_param_df.groupby(['group_id'])['param'].mean(),
    wrm_param_df.groupby(['group_id'])['param'].var(),
    wrm_param_df.groupby(['group_id'])['param'].std(),
], axis=1)
priority_df.columns=['sum', 'mean', 'var', 'std']
priority_df['m_s'] = priority_df['mean']/priority_df['std']
priority_df.sort_values('sum', ascending=False)

,sum,mean,var,std,m_s
group_id,,,,,
item_id_store_id_,0.661470,0.000022,8.288383e-10,0.000029,0.753560
item_id_state_id_,0.344453,0.000038,2.030212e-09,0.000045,0.835757
item_id_,0.190252,0.000062,4.733402e-09,0.000069,0.906954
store_id_dept_id_,0.008651,0.000124,2.609595e-09,0.000051,2.419215
store_id_cat_id_,0.004318,0.000144,1.654410e-09,0.000041,3.538401
state_id_dept_id_,0.003112,0.000148,2.876452e-09,0.000054,2.762661
state_id_cat_id_,0.001483,0.000165,1.637001e-09,0.000040,4.073359
store_id_,0.001470,0.000147,7.678679e-10,0.000028,5.303109
dept_id_,0.001212,0.000173,3.099091e-09,0.000056,3.109629


In [7]:
i=1
train_d_cols = d_cols[:-28*(i+1)]
valid_d_cols = d_cols[-28*(i+1):-28*i]
weight_d_cols = train_d_cols[-28:]
train_df = df[id_columns+train_d_cols]
val_df = df[id_columns+valid_d_cols]

weight_df = get_weight(
    train_df=train_df, weight_columns=weight_d_cols, 
    calendar=calendar_df, price_df=sell_prices_df, id_columns=id_columns
)

weight_data = {}
denominator = {}
name = []
for i, group_id in enumerate(tqdm(group_ids)):
    lv_weight = weight_df.groupby(group_id)[weight_d_cols].sum().sum(axis=1)
    weight_data.update((lv_weight / lv_weight.sum()).to_dict())
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
    for key, value in tmp_a.items() :
        value = np.array(list(value.values()))
        value = ((value[1:]-value[:-1])**2).mean()
        if value<=0:
            value=1
        a[key] = value
    denominator.update(a)
    
    s = str()
    for g in group_id:
        s+=f'{g}_'
    name+=[s for _ in range(len(a))]
    
    
    
    
weight_data = pd.DataFrame(pd.Series(weight_data))
weight_data.columns=['weight']
weight_data['group_id'] = name

denominator = pd.DataFrame(pd.Series(denominator))
denominator.columns=['denominator']
denominator = 1/denominator
denominator = np.sqrt(denominator)

wrm_param_df = pd.concat([weight_data, denominator], axis=1)
wrm_param_df['param'] = wrm_param_df.weight*wrm_param_df.denominator

wrm_param_df.sort_values('param', ascending=False, inplace=True)
    
    
priority_df = pd.concat([
    wrm_param_df.groupby(['group_id'])['param'].sum(),
    wrm_param_df.groupby(['group_id'])['param'].mean(),
    wrm_param_df.groupby(['group_id'])['param'].var(),
    wrm_param_df.groupby(['group_id'])['param'].std(),
], axis=1)
priority_df.columns=['sum', 'mean', 'var', 'std']
priority_df['m_s'] = priority_df['mean']/priority_df['std']
priority_df.sort_values('sum', ascending=False)

,sum,mean,var,std,m_s
group_id,,,,,
item_id_store_id_,0.665719,0.000022,9.290839e-10,0.000030,0.716319
item_id_state_id_,0.347165,0.000038,2.292546e-09,0.000048,0.792680
item_id_,0.190641,0.000063,5.332497e-09,0.000073,0.856234
store_id_dept_id_,0.008718,0.000125,2.792042e-09,0.000053,2.357088
store_id_cat_id_,0.004380,0.000146,1.890486e-09,0.000043,3.357840
state_id_dept_id_,0.003148,0.000150,3.098434e-09,0.000056,2.693442
state_id_cat_id_,0.001515,0.000168,1.997874e-09,0.000045,3.765485
store_id_,0.001470,0.000147,6.710423e-10,0.000026,5.675549
dept_id_,0.001222,0.000175,3.311945e-09,0.000058,3.033367


In [8]:
i=4
train_d_cols = d_cols[:-28*(i+1)]
valid_d_cols = d_cols[-28*(i+1):-28*i]
weight_d_cols = train_d_cols[-28:]
train_df = df[id_columns+train_d_cols]
val_df = df[id_columns+valid_d_cols]

weight_df = get_weight(
    train_df=train_df, weight_columns=weight_d_cols, 
    calendar=calendar_df, price_df=sell_prices_df, id_columns=id_columns
)

weight_data = {}
denominator = {}
name = []
for i, group_id in enumerate(tqdm(group_ids)):
    lv_weight = weight_df.groupby(group_id)[weight_d_cols].sum().sum(axis=1)
    weight_data.update((lv_weight / lv_weight.sum()).to_dict())
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
    for key, value in tmp_a.items() :
        value = np.array(list(value.values()))
        value = ((value[1:]-value[:-1])**2).mean()
        if value<=0:
            value=1
        a[key] = value
    denominator.update(a)
    
    s = str()
    for g in group_id:
        s+=f'{g}_'
    name+=[s for _ in range(len(a))]
    
    
    
    
weight_data = pd.DataFrame(pd.Series(weight_data))
weight_data.columns=['weight']
weight_data['group_id'] = name

denominator = pd.DataFrame(pd.Series(denominator))
denominator.columns=['denominator']
denominator = 1/denominator
denominator = np.sqrt(denominator)

wrm_param_df = pd.concat([weight_data, denominator], axis=1)
wrm_param_df['param'] = wrm_param_df.weight*wrm_param_df.denominator

wrm_param_df.sort_values('param', ascending=False, inplace=True)
    
    
priority_df = pd.concat([
    wrm_param_df.groupby(['group_id'])['param'].sum(),
    wrm_param_df.groupby(['group_id'])['param'].mean(),
    wrm_param_df.groupby(['group_id'])['param'].var(),
    wrm_param_df.groupby(['group_id'])['param'].std(),
], axis=1)
priority_df.columns=['sum', 'mean', 'var', 'std']
priority_df['m_s'] = priority_df['mean']/priority_df['std']
priority_df.sort_values('sum', ascending=False)

,sum,mean,var,std,m_s
group_id,,,,,
item_id_store_id_,0.693728,0.000023,1.130094e-09,0.000034,0.676822
item_id_state_id_,0.357184,0.000039,2.635372e-09,0.000051,0.760663
item_id_,0.197888,0.000065,6.161688e-09,0.000078,0.826822
store_id_dept_id_,0.009095,0.000130,2.668616e-09,0.000052,2.515225
store_id_cat_id_,0.004588,0.000153,2.219327e-09,0.000047,3.246298
state_id_dept_id_,0.003291,0.000157,2.837146e-09,0.000053,2.942529
state_id_cat_id_,0.001590,0.000177,2.892417e-09,0.000054,3.284388
store_id_,0.001507,0.000151,7.444414e-10,0.000027,5.521784
dept_id_,0.001305,0.000186,2.992836e-09,0.000055,3.406514


In [9]:
i=10
train_d_cols = d_cols[:-28*(i+1)]
valid_d_cols = d_cols[-28*(i+1):-28*i]
weight_d_cols = train_d_cols[-28:]
train_df = df[id_columns+train_d_cols]
val_df = df[id_columns+valid_d_cols]

weight_df = get_weight(
    train_df=train_df, weight_columns=weight_d_cols, 
    calendar=calendar_df, price_df=sell_prices_df, id_columns=id_columns
)

weight_data = {}
denominator = {}
name = []
for i, group_id in enumerate(tqdm(group_ids)):
    lv_weight = weight_df.groupby(group_id)[weight_d_cols].sum().sum(axis=1)
    weight_data.update((lv_weight / lv_weight.sum()).to_dict())
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
    for key, value in tmp_a.items() :
        value = np.array(list(value.values()))
        value = ((value[1:]-value[:-1])**2).mean()
        if value<=0:
            value=1
        a[key] = value
    denominator.update(a)
    
    s = str()
    for g in group_id:
        s+=f'{g}_'
    name+=[s for _ in range(len(a))]
    
    
    
    
weight_data = pd.DataFrame(pd.Series(weight_data))
weight_data.columns=['weight']
weight_data['group_id'] = name

denominator = pd.DataFrame(pd.Series(denominator))
denominator.columns=['denominator']
denominator = 1/denominator
denominator = np.sqrt(denominator)

wrm_param_df = pd.concat([weight_data, denominator], axis=1)
wrm_param_df['param'] = wrm_param_df.weight*wrm_param_df.denominator

wrm_param_df.sort_values('param', ascending=False, inplace=True)
    
    
priority_df = pd.concat([
    wrm_param_df.groupby(['group_id'])['param'].sum(),
    wrm_param_df.groupby(['group_id'])['param'].mean(),
    wrm_param_df.groupby(['group_id'])['param'].var(),
    wrm_param_df.groupby(['group_id'])['param'].std(),
], axis=1)
priority_df.columns=['sum', 'mean', 'var', 'std']
priority_df['m_s'] = priority_df['mean']/priority_df['std']
priority_df.sort_values('sum', ascending=False)

,sum,mean,var,std,m_s
group_id,,,,,
item_id_store_id_,0.738327,0.000024,1.352918e-09,0.000037,0.658348
item_id_state_id_,0.374717,0.000041,3.037670e-09,0.000055,0.743283
item_id_,0.208196,0.000068,7.014535e-09,0.000084,0.815295
store_id_dept_id_,0.009219,0.000132,3.216222e-09,0.000057,2.322311
store_id_cat_id_,0.004633,0.000154,2.613397e-09,0.000051,3.020824
state_id_dept_id_,0.003305,0.000157,3.617829e-09,0.000060,2.616383
state_id_cat_id_,0.001600,0.000178,3.120561e-09,0.000056,3.183237
store_id_,0.001512,0.000151,1.040966e-09,0.000032,4.687776
dept_id_,0.001302,0.000186,3.691409e-09,0.000061,3.062054


In [10]:
i=20
train_d_cols = d_cols[:-28*(i+1)]
valid_d_cols = d_cols[-28*(i+1):-28*i]
weight_d_cols = train_d_cols[-28:]
train_df = df[id_columns+train_d_cols]
val_df = df[id_columns+valid_d_cols]

weight_df = get_weight(
    train_df=train_df, weight_columns=weight_d_cols, 
    calendar=calendar_df, price_df=sell_prices_df, id_columns=id_columns
)

weight_data = {}
denominator = {}
name = []
for i, group_id in enumerate(tqdm(group_ids)):
    lv_weight = weight_df.groupby(group_id)[weight_d_cols].sum().sum(axis=1)
    weight_data.update((lv_weight / lv_weight.sum()).to_dict())
    
    tmp_a = train_df.groupby(group_id)[train_d_cols].sum().T.to_dict()
    a = {}
    for key, value in tmp_a.items() :
        value = np.array(list(value.values()))
        value = ((value[1:]-value[:-1])**2).mean()
        if value<=0:
            value=1
        a[key] = value
    denominator.update(a)
    
    s = str()
    for g in group_id:
        s+=f'{g}_'
    name+=[s for _ in range(len(a))]
    
    
    
    
weight_data = pd.DataFrame(pd.Series(weight_data))
weight_data.columns=['weight']
weight_data['group_id'] = name

denominator = pd.DataFrame(pd.Series(denominator))
denominator.columns=['denominator']
denominator = 1/denominator
denominator = np.sqrt(denominator)

wrm_param_df = pd.concat([weight_data, denominator], axis=1)
wrm_param_df['param'] = wrm_param_df.weight*wrm_param_df.denominator

wrm_param_df.sort_values('param', ascending=False, inplace=True)
    
    
priority_df = pd.concat([
    wrm_param_df.groupby(['group_id'])['param'].sum(),
    wrm_param_df.groupby(['group_id'])['param'].mean(),
    wrm_param_df.groupby(['group_id'])['param'].var(),
    wrm_param_df.groupby(['group_id'])['param'].std(),
], axis=1)
priority_df.columns=['sum', 'mean', 'var', 'std']
priority_df['m_s'] = priority_df['mean']/priority_df['std']
priority_df.sort_values('sum', ascending=False)

,sum,mean,var,std,m_s
group_id,,,,,
item_id_store_id_,0.734302,0.000024,1.224169e-09,0.000035,0.688330
item_id_state_id_,0.392684,0.000043,3.117036e-09,0.000056,0.768942
item_id_,0.219112,0.000072,7.580224e-09,0.000087,0.825407
store_id_dept_id_,0.009137,0.000131,3.316909e-09,0.000058,2.266381
store_id_cat_id_,0.004499,0.000150,2.372894e-09,0.000049,3.078932
state_id_dept_id_,0.003274,0.000156,2.926276e-09,0.000054,2.881915
store_id_,0.001556,0.000156,1.387007e-09,0.000037,4.176945
state_id_cat_id_,0.001537,0.000171,1.425479e-09,0.000038,4.522028
dept_id_,0.001301,0.000186,2.959726e-09,0.000054,3.417259
